<div style="border:solid green 2px; padding: 20px">
<b>Hello Daniel!</b>

My name is Evgeniy D. I'm going to review your project!

My main goal is not to show that any mistakes have been made somewhere, but to share my experience that will help you in your further work. Further in the file you can see my comments, try to take them into account when performing the next tasks.


You can find my comments in <font color='green'>green</font>, <font color='blue'>blue</font> or <font color='orange'>orange</font> boxes like this:

<div class="alert alert-success">
<b>Success:</b> if everything is done succesfully
</div>

<div class="alert alert-warning">
<b>Remarks: </b> if I can give some recommendations
</div>

<div class="alert alert-danger">
<b>Needs fixing:</b> if the block requires some corrections. Work can't be accepted with the red comments.
</div>


If you have 3 orange comments, we will need to adjust the project.
    
Let's work on the project in dialogue: if you change something in the project or respond to my comments, write about it. It will be easier for me to track the changes if you highlight your comments:
    
<div class="alert alert-info"> <b>Student comment:</b> For example like this.</div>

# Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Getting started is very important. So you explain what it is dedicated to. You can also provide a brief outline of the work itself and a description of the columns used.
    
</div>

## 1 Open the data file and have a look at the general information. 
***

In [1]:
import pandas as pd
import numpy as np
import os.path
from pandas.api.types import is_numeric_dtype
from IPython.display import display, display_html
from IPython.display import Markdown as md

In [2]:
def frequency(target):
    '''Returns a Series containing counts of unique rows, with percent
    
    Keyword argument:
    target -- a column name. defaults to 'df' variable.
    '''
    
    if target is None: target = df
    elif isinstance(target, str): target = df[target]
        
    concat_lambda = lambda dropna=None: pd.concat([
            target.value_counts(dropna=dropna).rename('count'),
            target.value_counts(normalize=True, dropna=dropna).rename('percent')
        ], axis=1, sort=True).sort_values('count', ascending=False)
        
    try:
        return concat_lambda(False)
    except Exception:
        return concat_lambda()

In [3]:
from itertools import combinations

# this function existence is only so the rest of the notebook looks cleaner.
# it is unmaintainable, and should be deleted,
# or at least refactored to a class and re-written.
def explore_column(column, 
                   round_to=3, 
                   sort='index', 
                   missing_values=True, 
                   apply=None, 
                   compare=False, 
                   display=True):
    '''
    Displays unique rows count (with percent) of a Dataframe's column/s.
    Can report missing values count. 
    Can also compare a list of columns, pair by pair.
    
    column (str/list)      -- column name / list of columns
    round_to (int)         -- decimal round of float values, 3 by default
    sort ('index'/'value') -- sort 'unique rows count' report by either 'index' (default), or 'value', or None
    missing values (bool)  -- report missing values count
    apply (function)       -- apply any transformation (function) to the resulting 'unique rows count' report
    compare (bool)         -- compare every pair of columns
    display (bool)         -- print resulting reports
    
    Returns:
    if a single column:
        dataframe: unique rows count report
        int: missing values count (optional)
        
    if multiple columns:
        list<dataframe>: unique rows count report, by column
        dict: missing values count, by column (optional)
        dict: comparison report, by pairs of columns (optional)
    '''
    
    def build_report(c):
        df2 = None
        if is_numeric_dtype(df[c]):
            df2 = df[c].describe().to_frame().astype('object').T
        
        df1 = frequency(c)
        
        if sort == 'index':
            df1.sort_index(inplace=True)
        elif sort == 'value' or sort == 'count':
            df1.sort_values('count', ascending=False, inplace=True)
            
        df1 = df1.rename_axis(c).round(round_to)
            
        if apply and callable(apply):
            df1 = apply(df1)
            
        df1 = df1.astype('object').T
        
        return (df2, df1) if isinstance(df2, pd.DataFrame) else df1
    
    def _compare(c1, c2):
        if len(c1.columns) != len(c2.columns):
            return False
        
        return (c1.copy().rename(columns=lambda x:'') \
            == c2.copy().rename(columns=lambda x:'')).all().all()
    
    def display_summaries(summaries):
        func = lambda s: _display(*s) if isinstance(s, list) or isinstance(s, tuple) else _display(s)
        
        if isinstance(summaries, list):
            for s in summaries:
                func(s)
        else:
            func(summaries)
            
    _display = globals()['display']
    
    if isinstance(column, list):
        summaries = [build_report(c) for c in column]
        output = [summaries]
        
        nans = None
        if missing_values:
            nans = {df[c].name: df[c].isna().sum() for c in column}
            output.append(nans)
        
        comparison = None
        if compare:
            comparison = {f'{c1.columns.name}__{c2.columns.name}': _compare(c1, c2) 
                               for c1, c2 in combinations(
                                   summaries[1] if isinstance(summaries, list) else summaries,2)}
            output.append(comparison)
            
        if display:
            display_summaries(summaries)
                
            if missing_values:
                _display(md(f'Missing values: {nans}'))
    else:
        summaries = build_report(column)
        output = [summaries]
        
        nans = None
        if missing_values:
            nans = df[column].isna().sum()
            output.append(nans)
                      
        if display:
            display_summaries(summaries)
                
            if missing_values:
                _display(md(f'Missing values: {nans}'))
    
    return output if len(output) > 1 else summaries

# examples
# a, b = explore_column('education', sort='count');
# print(type(a), type(b))
# a, b, c = explore_column(['education', 'education_id'], sort='count', compare=True);
# print(type(a), type(b), type(c))

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
It's great that we are already creating our own functions :)

It is especially cool that they have docstrings, because with the help of them it is much easier to understand what the function is doing.
    
</div>

## Inspect file
***
\* 'Dataframe' will be abbreviated 'df' for convenience.  
\* Most dataframes will be presented after being transposed, to save some space.

First, inspect the dataset file's metadata (size (mb), encoding, date),  
and print a small snippet.  

Afterwards,  
load the file into a pandas df,  
visually inspect the df itself and confirm it has loaded successfully,  
then its metadata (index, columns' name and type, memory).

In [4]:
# use relevant dataset file, depending on the platform the project runs on (local/review).

filename = 'credit_scoring_eng.csv'
filepath = f'/datasets/{filename}'
if not os.path.isfile(filepath):
    filepath = filename
# print(filepath)

In [5]:
!du -M "$filepath"          # filesize in MiB
!stat -l "$filepath"        # date, filename
!file "$filepath"           # file type, encoding
try:
    !head -n6 "$filepath"   # file preview
except Exception as e:
    print(e)

du: invalid option -- 'M'
Try 'du --help' for more information.
stat: invalid option -- 'l'
Try 'stat --help' for more information.
/bin/bash: file: command not found
children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1,-8437.673027760233,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,-4024.803753850451,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
0,-5623.422610230956,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,-4124.747206540018,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
0,340266.07204682194,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Interesting way 😊👏
    
</div>

In [6]:
df = pd.read_csv(filepath)
pd.options.display.max_columns = len(df.columns)

In [7]:
display(df.head(5))
print(f'index: {df.index}\nshape: {df.shape}\n')
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


index: RangeIndex(start=0, stop=21525, step=1)
shape: (21525, 12)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


First 5 rows of the dataframe are matching the first 5 rows of the file's preview, parsing looks OK.

In [8]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


In [9]:
int(401755 / 365)

1100

Various qunatitative columns show $\color{red}{\text{problems}}$:
- $\color{red}{\text{children}}$: negative -1, positive 20 (rare)
- $\color{red}{\text{days_employed}}$: negative values, max value exploding (~1100 years), missing values
- $\color{red}{\text{dob_years}}$: complex name
- $\color{red}{\text{total_income}}$: missing values, max >> 75th percentile, 100k scale while other percentiles are on the same scale (10k)

They will be examined more in detail in a later section.

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
We looked at our data in detail, let's prepare it now.
    
</div>

### Variable Types
#### Categorical / Quantitative
***

In [10]:
def variable_types(round_to=2):
    '''Displays a Series with counts of Variable Types (Quantitative / Categorical)'''
    display(
        _variable_types.to_frame('type').T,
        frequency(_variable_types).round(round_to))
    
_variable_types = pd.Series({
    'children': 'quantitative',
    'days_employed': 'quantitative',
    'dob_years': 'quantitative',
    'education': 'categorical',
    'education_id': 'id',
    'family_status': 'categorical',
    'family_status_id': 'id',
    'gender': 'categorical',
    'income_type': 'categorical',
    'debt': np.NaN,
    'total_income': 'quantitative',
    'purpose': np.NaN,
})

variable_types()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
type,quantitative,quantitative,quantitative,categorical,id,categorical,id,categorical,categorical,NaN,quantitative,NaN


,count,percent
categorical,4,0.33
quantitative,4,0.33
id,2,0.17
NaN,2,0.17


There are 2 columns of unclear variable type: 'debt' and 'purpose'. Exploring further.

In [11]:
df['debt'].value_counts(dropna=False)

0    19784
1     1741
Name: debt, dtype: int64

'debt' has 2 unique values: 0 and 1, a categorical variable.

In [12]:
_variable_types['debt'] = 'categorical'

In [13]:
frequency('purpose').round(2).astype('object').T

,wedding ceremony,having a wedding,to have a wedding,real estate transactions,buy commercial real estate,buying property for renting out,...,to get a supplementary education,education,getting an education,profile education,getting higher education,to become educated
count,797,777,774,676,664,653,...,447,447,443,436,426,412
percent,0.04,0.04,0.04,0.03,0.03,0.03,...,0.02,0.02,0.02,0.02,0.02,0.02


'purpose' is of 'string' data type, appears to be free-text, but from a data analysis point of view - it actually hides a categorical variable.  
There are apparent recurrent themes, i.e.: 'wedding', 'real estate', etc..  
Later on it will be processed and converted to categories.  
_conclusion_: categorical variable.

In [14]:
_variable_types['purpose'] = 'categorical'

In [15]:
display(md('<br />**Variable Types summary**'))
variable_types()

<br />**Variable Types summary**

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
type,quantitative,quantitative,quantitative,categorical,id,categorical,id,categorical,categorical,categorical,quantitative,categorical


,count,percent
categorical,6,0.50
quantitative,4,0.33
id,2,0.17


***
### Explore by column
Zooming in on columns, one by one, in order to detect more issues.

__Children__
***

In [16]:
explore_column('children');

,count,mean,std,min,25%,50%,75%,max
children,21525,0.538908,1.38159,-1,0,0,1,20


children,-1,0,1,2,3,4,5,20
count,47,14149,4818,2055,330,41,9,76
percent,0.002,0.657,0.224,0.095,0.015,0.002,0,0.004


Missing values: 0

Can't have a negative amount of children (-1), will treat it as a typing mistake - $\color{green}{\text{convert to 1}}$.  
To have 20 children is very rare, and 76 occurences (vs. 9 occurences of 5 children) is irrational.  
\- will treat it as a typing mistake as well, and $\color{green}{\text{convert to 2}}$.

__Days Employed__
***

In [17]:
explore_column('days_employed');

,count,mean,std,min,25%,50%,75%,max
days_employed,19351,63046.5,140827,-18388.9,-2747.42,-1203.37,-291.096,401755


days_employed,-18388.949901,-17615.563266,-16593.472817,-16264.699501,-16119.687737,-15835.725775,...,401663.850046,401674.466633,401675.093434,401715.811749,401755.400475,NaN
count,1,1,1,1,1,1,...,1,1,1,1,1,2174
percent,0,0,0,0,0,0,...,0,0,0,0,0,0.101


Missing values: 2174

In [18]:
print(f'401755 (max days_employed value) / 365 days = {401755 / 365} years')
print(f'80 years * 365 days = {80 * 365} days')

401755 (max days_employed value) / 365 days = 1100.6986301369864 years
80 years * 365 days = 29200 days


In [19]:
days_employed_bins = pd.cut(
    df['days_employed'],
    bins=[-40000, 0, 100, 1000, 10000, 30000, 1000000]
).value_counts().sort_index(ascending=False)
days_employed_bins

(30000, 1000000]     3445
(10000, 30000]          0
(1000, 10000]           0
(100, 1000]             0
(0, 100]                0
(-40000, 0]         15906
Name: days_employed, dtype: int64

In [20]:
days_employed_bins

(30000, 1000000]     3445
(10000, 30000]          0
(1000, 10000]           0
(100, 1000]             0
(0, 100]                0
(-40000, 0]         15906
Name: days_employed, dtype: int64

In [21]:
print(f'days employed of range (30000, 1000000]: {days_employed_bins.iloc[0] / days_employed_bins.sum():.0%}')
print(f'days employed of range (-40000, 0]: {days_employed_bins.iloc[-1] / days_employed_bins.sum():.0%}')

days employed of range (30000, 1000000]: 18%
days employed of range (-40000, 0]: 82%


<div class="alert alert-danger"> <b>Reviewer comment:</b> 
    
<s>Something went wrong ☹️, please see what's wrong
    
</div>

In [22]:
pd.__version__

'0.25.1'

<div class="alert alert-info"> <b>Student comment:</b> <br /><br />  
Apparently my local pandas version is '1.2.1', but this cloud version is '0.25.1' 😨 <br />
days_employed_bins[0] is what was causing the recursion, changed to iloc[0].  

Also, I've realized I should have run the notebook in the cloud before submitting,  
did it now just to make sure we don't encouter any more exceptions. cheers!
</div>

<div class="alert alert-success"> <b>Reviewer comment (2):</b> 
    
Yeah, so it is ((

By the way, if you are using a newer version, you can always update any library before executing the project.

With the following command
    
`!pip install --upgrade pandas` 
    
</div>

100% of the data appears to be a erroneous:  
- 18% 80-1100 years
- 82% negative  

Try to recover the data, by contacting the person responsible for the data.
If the data can't be recovered,  
'days_employed' should not impact our main goal of assessing a person's abillity to repay a loan,  
there are stronger factors available (i.e. income_total, children, family_status).

'days_employed' can be $\color{green}{\text{removed}}$ altogether.

__Age__
***

In [23]:
explore_column('dob_years');

,count,mean,std,min,25%,50%,75%,max
dob_years,21525,43.2934,12.5746,0,33,42,53,75


dob_years,0,19,20,21,22,23,...,70,71,72,73,74,75
count,101,14,51,111,183,254,...,65,58,33,8,6,1
percent,0.005,0.001,0.002,0.005,0.009,0.012,...,0.003,0.003,0.002,0,0,0


Missing values: 0

Age can't be 0 in this context (bank loan), it should be $\color{green}{\text{replaced with median}}$, and the column's $\color{green}{\text{name}}$ should be changed to '$\color{green}{\text{age}}$'.

__Education__
***

In [24]:
_, _, comparisons = explore_column(['education', 'education_id'], compare=True)
print(f'Do the tables match? {list(comparisons.values())[0]}');

education,BACHELOR'S DEGREE,Bachelor's Degree,GRADUATE DEGREE,Graduate Degree,PRIMARY EDUCATION,Primary Education,...,Some College,bachelor's degree,graduate degree,primary education,secondary education,some college
count,274,268,1,1,17,15,...,47,4718,4,250,13750,668
percent,0.013,0.012,0,0,0.001,0.001,...,0.002,0.219,0,0.012,0.639,0.031


,count,mean,std,min,25%,50%,75%,max
education_id,21525,0.817236,0.548138,0,1,1,1,4


education_id,0,1,2,3,4
count,5260,15233,744,282,6
percent,0.244,0.708,0.035,0.013,0


Missing values: {'education': 0, 'education_id': 0}

Do the tables match? False


'education' has duplicates with different styling (letter case) - values should be normalized to $\color{green}{\text{lowercase}}$,  
furthermore 'education' and 'education_id' value counts should coincide.

__Family status__
***

In [25]:
_, _, comparisons = explore_column([
    'family_status',
    'family_status_id'
], sort='values', compare=True)

print(f'Do the tables match? {list(comparisons.values())[0]}')

family_status,married,civil partnership,unmarried,divorced,widow / widower
count,12380,4177,2813,1195,960
percent,0.575,0.194,0.131,0.056,0.045


,count,mean,std,min,25%,50%,75%,max
family_status_id,21525,0.972544,1.42032,0,0,0,1,4


family_status_id,0,1,4,3,2
count,12380,4177,2813,1195,960
percent,0.575,0.194,0.131,0.056,0.045


Missing values: {'family_status': 0, 'family_status_id': 0}

Do the tables match? False


Family status column appears to be in order, $\color{green}{\text{OK}}$.

__Gender__
***

In [26]:
explore_column('gender', round_to=6);

gender,F,M,XNA
count,14236,7288,1
percent,0.66137,0.338583,4.6e-05


Missing values: 0

Gender column has a single outlier 'XNA'.  
Less than 0.01% is statistically insignificant,  
value can be replaced with either 'F' or 'M' for convenience, or $\color{green}{\text{ignored}}$ altogether.

__Income type__
***

In [27]:
explore_column('income_type', sort='value');

income_type,employee,business,retiree,civil servant,entrepreneur,unemployed,paternity / maternity leave,student
count,11119,5085,3856,1459,2,2,1,1
percent,0.517,0.236,0.179,0.068,0,0,0,0


Missing values: 0

income_type should have a complimentary 'income_type_$\color{green}{\text{id}}$' $\color{green}{\text{column}}$.  

__Debt__
***

In [28]:
explore_column('debt');

,count,mean,std,min,25%,50%,75%,max
debt,21525,0.0808827,0.272661,0,0,0,0,1


debt,0,1
count,19784,1741
percent,0.919,0.081


Missing values: 0

Debt is of logical type (a categorical subtype).  
Should be $\color{green}{\text{converted to Boolean}}$ data type.

__Total income__
***

In [29]:
explore_column('total_income');

,count,mean,std,min,25%,50%,75%,max
total_income,19351,26787.6,16475.5,3306.76,16488.5,23202.9,32549.6,362497


total_income,3306.762,3392.845,3418.824,3471.216,3503.298,3595.641,...,273809.483,274402.943,276204.162,352136.354,362496.645,NaN
count,1,1,1,1,1,1,...,1,1,1,1,1,2174
percent,0,0,0,0,0,0,...,0,0,0,0,0,0.101


Missing values: 2174

Can be $\color{green}{\text{renamed}}$ '$\color{green}{\text{income}}$'.

'total_income' has $\color{red}{\text{missing values}}$.  
Income is critical in assessing a person's ability to repay a loan, and the data should be recovered if possible.

__Purpose__
***

In [30]:
explore_column('purpose');

purpose,building a property,building a real estate,buy commercial real estate,buy real estate,buy residential real estate,buying a second-hand car,...,to have a wedding,to own a car,transactions with commercial real estate,transactions with my real estate,university education,wedding ceremony
count,620,626,664,624,607,479,...,774,480,651,630,453,797
percent,0.029,0.029,0.031,0.029,0.028,0.022,...,0.036,0.022,0.03,0.029,0.021,0.037


Missing values: 0

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Also we can to use transpose for our dataframe.
    
To improve the readability of the data.
    
</div>

In [106]:
explore_column('income_type')[0].T

income_type,business,civil servant,employee,entrepreneur,paternity / maternity leave,retiree,student,unemployed
count,5085,1459,11119,2,1,3856,1,2
percent,0.236,0.068,0.517,0,0,0.179,0,0


Missing values: 0

,count,percent
income_type,,
business,5085,0.236
civil servant,1459,0.068
employee,11119,0.517
entrepreneur,2,0
paternity / maternity leave,1,0
retiree,3856,0.179
student,1,0
unemployed,2,0


Purpose appears to be free-text and have duplicates of different styling but the same meaning,  
and in fact can and should be processed and $\color{green}{\text{converted to a category}}$.

### 1.1 Conclusion

Dataset has both categorical and quantitative variable types.  
There are issues with the dataset, which will be addressed after the initial analyses.  

List of detected issues:
- [ ]  children: negative values, inflated values
- [ ]  days_employed: drop
- [ ]  dob_years: column name, 0 value, categorize & extract
- [ ]  education: duplicates, categorize & extract
- [ ]  family_status: extract category
- [x]  gender: ~~single outlier~~
- [ ]  income_type: column name, categorize & extract
- [ ]  debt: wrong data type
- [ ]  total_income: column name, missing values, wrong data type
- [ ]  purpose: duplicates, categorize & extract

\* gender: a single outlier (which consists as less than 0.01%) of an insignificant variable can safely be ignored.

## 2 Data preprocessing
***

First deal with columns that don't have dependencies on other columns.

__Children__
***

Replacing values '-1' and '20'.

In [31]:
df.loc[df['children'] == -1, 'children'] = 1
df.loc[df['children'] == 20, 'children'] = 2

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
And if we looked at the fraction of these values, would we be able to make a decision in the direction of getting rid of this data?
    
</div>

In [32]:
explore_column('children');

,count,mean,std,min,25%,50%,75%,max
children,21525,0.479721,0.755528,0,0,0,1,5


children,0,1,2,3,4,5
count,14149,4865,2131,330,41,9
percent,0.657,0.226,0.099,0.015,0.002,0


Missing values: 0

__Days employed__
***

In [33]:
df = df.drop('days_employed', axis=1)

__Age__
***

Renaming 'dob_years' to 'age'.  
Replacing 0 value with median.

In [34]:
df['age'] = df['dob_years']
del df['dob_years']
df = df.reindex(columns=sorted(df.columns))

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
For delete columns from dataset, I recommend to use method `drop`, because method `del` used by when we need to delete varible from our RAM.
    
</div>

And if we looked at the fraction of these values, would we be able to make a decision in the direction of getting rid of this data?

In [35]:
_variable_types['age'] = _variable_types.pop('dob_years')

In [36]:
df['age'].agg(['mean', 'median'])

mean      43.29338
median    42.00000
Name: age, dtype: float64

In [37]:
df['age'].replace(0, int(df['age'].median()), inplace=True)

In [38]:
explore_column('age');

,count,mean,std,min,25%,50%,75%,max
age,21525,43.4905,12.2186,19,34,42,53,75


age,19,20,21,22,23,24,...,70,71,72,73,74,75
count,14,51,111,183,254,264,...,65,58,33,8,6,1
percent,0.001,0.002,0.005,0.009,0.012,0.012,...,0.003,0.003,0.002,0,0,0


Missing values: 0

__Education__
***

Transforming values to lower case.

In [39]:
df['education'] = df['education'].str.casefold()

In [40]:
explore_column(['education', 'education_id'], compare=True, sort='count');

education,secondary education,bachelor's degree,some college,primary education,graduate degree
count,15233,5260,744,282,6
percent,0.708,0.244,0.035,0.013,0


,count,mean,std,min,25%,50%,75%,max
education_id,21525,0.817236,0.548138,0,1,1,1,4


education_id,1,0,2,3,4
count,15233,5260,744,282,6
percent,0.708,0.244,0.035,0.013,0


Missing values: {'education': 0, 'education_id': 0}

'education' and 'education_id' values coincide.

### 2.1 Processing missing values

In [41]:
def missing_values_summary(df=df, round_to=2):
    '''Returns a missing values report'''
    
    na_count = df.isna().sum()
    total_size_per_column = na_count + df.count()
    
    report = pd.concat([
        na_count.astype('object').rename('na count'),
        total_size_per_column.astype('object').rename('column size'),
        (na_count / total_size_per_column).round(round_to).rename('percent'),
        _variable_types.rename('type')
    ], axis=1, sort=True).loc[df.columns]

    return report[report['na count'] != 0]
    
missing_values_summary()

,na count,column size,percent,type
total_income,2174,21525,0.1,quantitative


__Income - missing values__
***

In [42]:
df[df['total_income'].isna()].head(3)

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,purpose,total_income
12,65,0,0,secondary education,1,civil partnership,1,M,retiree,to have a wedding,NaN
26,41,0,0,secondary education,1,married,0,M,civil servant,education,NaN
29,63,0,0,secondary education,1,unmarried,4,F,retiree,building a real estate,NaN


In [43]:
df[~df['total_income'].isna()].head(3)

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,purpose,total_income
0,42,1,0,bachelor's degree,0,married,0,F,employee,purchase of the house,40620.102
1,36,1,0,secondary education,1,married,0,F,employee,car purchase,17932.802
2,33,0,0,secondary education,1,married,0,M,employee,purchase of the house,23341.752


Income depends on various factors, such as income_type, education, age, and even # of children.  
Data will be split to groups of education level and age (usually analogous to seniority),  
and then the missing values filled using the appropriate group.

\* To establish correlation, it's better to plot the data.

In [44]:
def group_by_age(row):
    '''Returns a label of the appropriate ages range'''
    
    age = row['age']
    
    if not pd.api.types.is_number(age):
        return ''
    
    if age < 20:
        return '0-19'
    elif age < 65:
        tens = age // 10
        return f'{tens}0-{tens}9'
    else:
        return '65+'

In [45]:
df_age_group = df.join(df.apply(group_by_age, axis=1).rename('age_group'))
df_age_group = df_age_group.reindex(columns=sorted(df_age_group))

In [46]:
education_labels_sorted = ['primary education', 'secondary education', 'some college', 'bachelor\'s degree', 'graduate degree']
df_education_age_pivot = df_age_group.pivot_table(
    index='age_group',
    columns='education',
    values='total_income',
    aggfunc='median'
).replace(np.NaN, 0).astype('int').reindex(columns=education_labels_sorted)
display(df_education_age_pivot)
display(md('_Total income by education and age_'))

education,primary education,secondary education,some college,bachelor's degree,graduate degree
age_group,,,,,
0-19,0,15761,14575,0,0
20-29,25488,21166,22698,25956,0
30-39,19542,22912,28463,28794,18187
40-49,21511,22960,29323,30094,31771
50-59,16922,21245,21132,28152,42945
60-69,18254,19467,26346,25796,40868
65+,15354,17943,27938,23353,15800


_Total income by education and age_

In [47]:
df_income_na = df_age_group.loc[
    df_age_group['total_income'].isna(),
    ['age_group', 'education']
]
df_income_filled = df_income_na.agg(
    lambda row:df_education_age_pivot.loc[row['age_group'], row['education']],
    axis=1
)
df['total_income'].fillna(df_income_filled, inplace=True)

In [48]:
missing_values_summary()

,na count,column size,percent,type


Missing values filled.

### 2.2 Conclusion

Missing data was detected for customer's income (a quantitative variable type), which is dependant on multiple factors,  
and was filled by taking the median of the relevant 'age + education' group.

\* Checklist:
- [x]  children: ~~negative values~~, ~~inflated values~~
- [x]  ~~days_employed: dropped~~
- [ ]  age: ~~column name~~, ~~0 value~~, categorize & extract
- [ ]  education: ~~letter case~~, ~~mismatch with id count~~, extract category
- [ ]  family_status: extract category
- [x]  gender: ~~single outlier~~
- [ ]  income_type: column name, categorize & extract
- [ ]  debt: wrong data type
- [ ]  total_income: ~~missing values~~, wrong data type
- [ ]  purpose: duplicates, categorize & extract

### 2.3 Data type replacement

__Debt - data type__
***

In [49]:
explore_column('debt');
df['debt'].dtype

,count,mean,std,min,25%,50%,75%,max
debt,21525,0.0808827,0.272661,0,0,0,0,1


debt,0,1
count,19784,1741
percent,0.919,0.081


Missing values: 0

dtype('int64')

In [50]:
df['debt'] = df['debt'].astype('bool')
explore_column('debt');
df['debt'].dtype

,count,unique,top,freq
debt,21525,2,False,19784


debt,False,True
count,19784,1741
percent,0.919,0.081


Missing values: 0

dtype('bool')

__Total income__
***

In [51]:
df['income'] = df['total_income'].astype('int')
del df['total_income']
explore_column('income');
df['income'].dtype

,count,mean,std,min,25%,50%,75%,max
income,21525,26452.4,15687.1,3306,17234,22960,31286,362496


income,3306,3392,3418,3471,3503,3595,...,255618,273809,274402,276204,352136,362496
count,1,1,1,1,1,1,...,1,1,1,1,1,1
percent,0,0,0,0,0,0,...,0,0,0,0,0,0


Missing values: 0

dtype('int64')

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Including the `fillna` and` astype` methods can be applied to multiple columns at the same time.
    
If there are more than 2 columns, then I recommend using the `for` loop.
    
Example:
    
`df.astype ({" col1 ":" float64 "," col2 ":" int64 "})`
    
</div>

In [52]:
_variable_types['income'] = _variable_types.pop('total_income')

In [53]:
df.head(3)

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,purpose,income
0,42,1,False,bachelor's degree,0,married,0,F,employee,purchase of the house,40620
1,36,1,False,secondary education,1,married,0,F,employee,car purchase,17932
2,33,0,False,secondary education,1,married,0,M,employee,purchase of the house,23341


### 2.4 Conclusion

* 'debt' was converted from int to boolean for semantic purposes.
* 'total_income' was renamed 'income' and converted from float64 to int64,  
since a fraction of single currency unit is meaningless for our purposes,  
and int is easier for the processor to work with.

\* Checklist:
- [x]  children: ~~negative values~~, ~~inflated values~~
- [x]  ~~days_employed: dropped~~
- [ ]  age: ~~column name~~, ~~0 value~~, categorize & extract
- [ ]  education: ~~letter case~~, ~~mismatch with id count~~, extract category
- [ ]  family_status: extract category
- [x]  gender: ~~single outlier~~
- [ ]  income: ~~column name~~, categorize & extract
- [x]  debt: ~~wrong data type~~
- [x]  total_income: ~~missing values~~, ~~wrong data type~~
- [ ]  purpose: duplicates, categorize & extract

### 2.5 Processing duplicates

In [54]:
df[df.duplicated()].head(5)

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,purpose,income
2849,41,0,False,secondary education,1,married,0,F,employee,purchase of the house for my family,22960
3290,58,0,False,secondary education,1,civil partnership,1,F,retiree,to have a wedding,21245
4182,34,1,False,bachelor's degree,0,civil partnership,1,F,employee,wedding ceremony,28794
4851,60,0,False,secondary education,1,civil partnership,1,F,retiree,wedding ceremony,19467
5557,58,0,False,secondary education,1,civil partnership,1,F,retiree,to have a wedding,21245


Searching for duplicates by hand doesn't return any useful results mainly because the rows don't have an associated id,  
so tracing duplicates is virtually impossible.  
Assume each row is a unique customer (and ask the person responsible for the dataset just to make sure).

In [55]:
explore_column('purpose');

purpose,building a property,building a real estate,buy commercial real estate,buy real estate,buy residential real estate,buying a second-hand car,...,to have a wedding,to own a car,transactions with commercial real estate,transactions with my real estate,university education,wedding ceremony
count,620,626,664,624,607,479,...,774,480,651,630,453,797
percent,0.029,0.029,0.031,0.029,0.028,0.022,...,0.036,0.022,0.03,0.029,0.021,0.037


Missing values: 0

In [56]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pandas.core.common import flatten
from collections import Counter

wordnet_lemma = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def extract_lemmas(sentence):
    words = nltk.word_tokenize(sentence)
    lemmas = [wordnet_lemma.lemmatize(w, pos='n') for w in words if w not in stop_words]
    return lemmas

<div class="alert alert-warning"> <b>Reviewer comment:</b> 
    
In best practies to import all the required libraries at the very beginning of the project.
    
This is a generally accepted format in programming, because it makes it easier to understand which packages will be used in our work.
    
</div>

In [57]:
purpose_lemmas = df['purpose'].map(extract_lemmas)
purpose_lemmas = pd.Series(Counter(flatten(purpose_lemmas.values))).sort_values(ascending=False)
purpose_lemmas.to_frame()

,0
real,4478
estate,4478
car,4315
purchase,3314
education,3114
transaction,2610
property,2542
buy,2367
wedding,2348
housing,1912


In [58]:
df.loc[df['purpose'].str.contains('renov'), 'purpose'].head(3)

139    housing renovation
163    housing renovation
228    housing renovation
Name: purpose, dtype: object

In [59]:
df.loc[df['purpose'].str.contains('residential'), 'purpose'].head(3)

14     buy residential real estate
96     buy residential real estate
116    buy residential real estate
Name: purpose, dtype: object

In [60]:
def categorize_purpose(purpose):
    lemmas = extract_lemmas(purpose)
    
    def assign_category(lemma):
        if lemma == 'wedding':
            return 'wedding'
        elif lemma == 'car':
            return 'vehicle'
        elif lemma in ['education', 'educated', 'university']:
            return 'education'
        elif lemma in ['housing', 'house', 'renovation', 'renovation']:
            return 'home'
        elif lemma in ['estate', 'real', 'property', 'building', 'construction', 'residential']:
            return 'real-estate'
        
    for lemma in lemmas:
        category = assign_category(lemma)
        if category:
            return category
    
    print(f'category not assigned: {lemma}')

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Loan categories have been identified correctly 👍
    
</div>

In [61]:
df['purpose'].head(5).to_frame().join(
    df['purpose'].head(5).apply(categorize_purpose).rename('purpose_categorized'))

,purpose,purpose_categorized
0,purchase of the house,home
1,car purchase,vehicle
2,purchase of the house,home
3,supplementary education,education
4,to have a wedding,wedding


In [62]:
df_temp = df.drop('purpose', axis=1).join(df['purpose'].apply(categorize_purpose))
df_temp.head()

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,income,purpose
0,42,1,False,bachelor's degree,0,married,0,F,employee,40620,home
1,36,1,False,secondary education,1,married,0,F,employee,17932,vehicle
2,33,0,False,secondary education,1,married,0,M,employee,23341,home
3,32,3,False,secondary education,1,married,0,M,employee,42820,education
4,53,0,False,secondary education,1,civil partnership,1,F,retiree,25378,wedding


In [63]:
df_temp.loc[df_temp['purpose'].isna(), 'purpose']

Series([], Name: purpose, dtype: object)

In [64]:
df = df_temp
del df_temp

### 2.6 Conclusion

In general, there isn't enought data (i.e. customer's id) to conclude that any rows are duplicates.  

There were duplicated entries for the 'education' column, which were handled by casefolding (lowercase) in a previous section.  

'purpose' column hid duplicated values, which were obscured by 'free-form' text.  
Those values were reduced to keywords (categories) using lemmatization.

### 2.7 Categorizing Data

In [65]:
def invert_series(series):
    '''
    Swap index and values of a pandas' series.
    '''
    
    return pd.Series(
        data=series.index,
        index=series.values
    ).rename_axis(series.name)

__Age__
***
Utilizing 'age_group' column, created in a previous secrion.

In [66]:
age_group_ref = pd.Series(
    data=sorted(df_age_group['age_group'].value_counts().index.values),
    name='age_group'
).rename_axis('age_group_id')
age_group_ref.to_frame().T

age_group_id,0,1,2,3,4,5,6
age_group,0-19,20-29,30-39,40-49,50-59,60-69,65+


In [67]:
age_group_ref_inverted = invert_series(age_group_ref)
age_group_ref_inverted.to_frame().T

age_group,0-19,20-29,30-39,40-49,50-59,60-69,65+
age_group_id,0,1,2,3,4,5,6


In [68]:
display(df.head(3), md("_'before' sample_"))

,age,children,debt,education,education_id,family_status,family_status_id,gender,income_type,income,purpose
0,42,1,False,bachelor's degree,0,married,0,F,employee,40620,home
1,36,1,False,secondary education,1,married,0,F,employee,17932,vehicle
2,33,0,False,secondary education,1,married,0,M,employee,23341,home


_'before' sample_

In [69]:
df_temp = df.join(df_age_group['age_group'] \
                  .apply(lambda v:age_group_ref_inverted[v]) \
                  .rename('age_group_id')) \
            .drop('age', axis=1)
df_temp = df_temp.reindex(columns=sorted(df_temp.columns))

display(
    df_temp.head(3),
    md(f"_'after' sample<br />shape: {df_temp.shape}_"),
    age_group_ref.to_frame().T
)

,age_group_id,children,debt,education,education_id,family_status,family_status_id,gender,income,income_type,purpose
0,3,1,False,bachelor's degree,0,married,0,F,40620,employee,home
1,2,1,False,secondary education,1,married,0,F,17932,employee,vehicle
2,2,0,False,secondary education,1,married,0,M,23341,employee,home


_'after' sample<br />shape: (21525, 11)_

age_group_id,0,1,2,3,4,5,6
age_group,0-19,20-29,30-39,40-49,50-59,60-69,65+


In [70]:
df = df_temp
del df_temp

__Education__
***

In [71]:
education_ref = df[['education', 'education_id']].drop_duplicates().set_index('education_id')
education_ref.T

education_id,0,1,2,3,4
education,bachelor's degree,secondary education,some college,primary education,graduate degree


Re-assigning ids to education categories, based on degree importance (0 - least important, 4 - most important).

In [72]:
education_ref = pd.Series(
    data = [
        'primary education',
        'secondary education',
        'some college',
        'bachelor\'s degree',
        'graduate degree'],
    name = 'education'
).rename_axis('education_id')
education_ref.to_frame().T

education_id,0,1,2,3,4
education,primary education,secondary education,some college,bachelor's degree,graduate degree


In [73]:
education_ref_inverted = invert_series(education_ref)
education_ref_inverted.to_frame().T

education,primary education,secondary education,some college,bachelor's degree,graduate degree
education_id,0,1,2,3,4


In [74]:
display(df.head(3), md("'before' sample"))

,age_group_id,children,debt,education,education_id,family_status,family_status_id,gender,income,income_type,purpose
0,3,1,False,bachelor's degree,0,married,0,F,40620,employee,home
1,2,1,False,secondary education,1,married,0,F,17932,employee,vehicle
2,2,0,False,secondary education,1,married,0,M,23341,employee,home


'before' sample

In [75]:
df_temp = df.drop('education_id', axis=1) \
            .join(df['education'] \
                  .apply(lambda v:education_ref_inverted[v]) \
                  .rename('education_id')) \
            .drop('education', axis=1)
df_temp = df_temp.reindex(columns=sorted(df_temp.columns))

display(
    df_temp.head(3),
    md(f"_'after' sample<br />shape: {df_temp.shape}_"),
    education_ref.to_frame().T
)

,age_group_id,children,debt,education_id,family_status,family_status_id,gender,income,income_type,purpose
0,3,1,False,3,married,0,F,40620,employee,home
1,2,1,False,1,married,0,F,17932,employee,vehicle
2,2,0,False,1,married,0,M,23341,employee,home


_'after' sample<br />shape: (21525, 10)_

education_id,0,1,2,3,4
education,primary education,secondary education,some college,bachelor's degree,graduate degree


In [76]:
df = df_temp
del df_temp

__Family status__
***

In [77]:
family_status_ref = df[['family_status', 'family_status_id']] \
                    .drop_duplicates() \
                    .set_index('family_status_id') \
                    .squeeze()
family_status_ref.to_frame().T

family_status_id,0,1,2,3,4
family_status,married,civil partnership,widow / widower,divorced,unmarried


In [78]:
display(df.head(3), md("'before' sample"))

,age_group_id,children,debt,education_id,family_status,family_status_id,gender,income,income_type,purpose
0,3,1,False,3,married,0,F,40620,employee,home
1,2,1,False,1,married,0,F,17932,employee,vehicle
2,2,0,False,1,married,0,M,23341,employee,home


'before' sample

In [79]:
df.drop('family_status', axis=1, inplace=True)
display(
    df.head(3),
    md(f"_'after' sample<br />shape: {df.shape}_"),
    family_status_ref.to_frame().T
)

,age_group_id,children,debt,education_id,family_status_id,gender,income,income_type,purpose
0,3,1,False,3,0,F,40620,employee,home
1,2,1,False,1,0,F,17932,employee,vehicle
2,2,0,False,1,0,M,23341,employee,home


_'after' sample<br />shape: (21525, 9)_

family_status_id,0,1,2,3,4
family_status,married,civil partnership,widow / widower,divorced,unmarried


__Income type__
***

In [80]:
income_type_ref = pd.Series(
    data=df['income_type'].value_counts().index,
    name='income_type',
).rename_axis('income_type_id')
income_type_ref.to_frame().T

income_type_id,0,1,2,3,4,5,6,7
income_type,employee,business,retiree,civil servant,entrepreneur,unemployed,student,paternity / maternity leave


In [81]:
income_type_ref_inverted = invert_series(income_type_ref)
income_type_ref_inverted.to_frame().T

income_type,employee,business,retiree,civil servant,entrepreneur,unemployed,student,paternity / maternity leave
income_type_id,0,1,2,3,4,5,6,7


In [82]:
display(df.head(3), md("_'before' sample_"))

,age_group_id,children,debt,education_id,family_status_id,gender,income,income_type,purpose
0,3,1,False,3,0,F,40620,employee,home
1,2,1,False,1,0,F,17932,employee,vehicle
2,2,0,False,1,0,M,23341,employee,home


_'before' sample_

In [83]:
df_temp = df.join(df['income_type'] \
                  .apply(lambda v:income_type_ref_inverted[v]) \
                  .rename('income_type_id')) \
            .drop('income_type', axis=1)
# df_temp = df_temp.reindex(columns=sorted(df_temp.columns))

display(
    df_temp.head(3),
    md(f"_'after' sample<br />shape: {df_temp.shape}_"),
    income_type_ref.to_frame().T
)

,age_group_id,children,debt,education_id,family_status_id,gender,income,purpose,income_type_id
0,3,1,False,3,0,F,40620,home,0
1,2,1,False,1,0,F,17932,vehicle,0
2,2,0,False,1,0,M,23341,home,0


_'after' sample<br />shape: (21525, 9)_

income_type_id,0,1,2,3,4,5,6,7
income_type,employee,business,retiree,civil servant,entrepreneur,unemployed,student,paternity / maternity leave


__Purpose__
***

In [84]:
purpose_ref = pd.Series(
    data=df['purpose'].value_counts().index,
    name='purpose',
).rename_axis('purpose_id')
purpose_ref.to_frame().T

purpose_id,0,1,2,3,4
purpose,real-estate,vehicle,education,home,wedding


In [85]:
purpose_ref_inverted = invert_series(purpose_ref)
purpose_ref_inverted.to_frame().T

purpose,real-estate,vehicle,education,home,wedding
purpose_id,0,1,2,3,4


In [86]:
display(df.head(3), md("_'before' sample_"))

,age_group_id,children,debt,education_id,family_status_id,gender,income,income_type,purpose
0,3,1,False,3,0,F,40620,employee,home
1,2,1,False,1,0,F,17932,employee,vehicle
2,2,0,False,1,0,M,23341,employee,home


_'before' sample_

In [87]:
df_temp = df.join(df['purpose'] \
                  .apply(lambda v:purpose_ref_inverted[v]) \
                  .rename('purpose_id')) \
            .drop('purpose', axis=1)
# df_temp = df_temp.reindex(columns=sorted(df_temp.columns))

display(
    df_temp.head(3),
    md(f"_'after' sample<br />shape: {df_temp.shape}_"),
    purpose_ref.to_frame().T
)

,age_group_id,children,debt,education_id,family_status_id,gender,income,income_type,purpose_id
0,3,1,False,3,0,F,40620,employee,3
1,2,1,False,1,0,F,17932,employee,1
2,2,0,False,1,0,M,23341,employee,3


_'after' sample<br />shape: (21525, 9)_

purpose_id,0,1,2,3,4
purpose,real-estate,vehicle,education,home,wedding


In [88]:
df = df_temp
del df_temp

In [89]:
df.head(1)

,age_group_id,children,debt,education_id,family_status_id,gender,income,income_type,purpose_id
0,3,1,False,3,0,F,40620,employee,3


### 2.8 Conclusion

Data preprocessing phase is done.

Various categorical variables were detected and extracted to standalone dataframes, 
keeping ids in both dataframes - new and old,  
in order to be able to link back the category name to the appropriate category id  
\- aka a relationship (a standard data practice, used extensively in databases).

'age' was converted to age groups and extracted,  
'education' was re-ordered by importance, reassigned ids and extracted,  
'family status' was extracted,  
'purpose' was converted to a category by the process of lemmatization (keyword extraction essentially).

\* Checklist:
- [x]  children: ~~negative values~~, ~~inflated values~~
- [x]  ~~days_employed: dropped~~
- [x]  age: ~~column name~~, ~~0 value~~, ~~categorize & extract~~
- [x]  education: ~~letter case~~, ~~mismatch with id count~~, ~~extract category~~
- [x]  family_status: ~~extract category~~
- [x]  gender: ~~single outlier~~
- [x]  income: ~~column name~~, ~~categorize & extract~~
- [x]  debt: ~~wrong data type~~
- [x]  total_income: ~~column name~~, ~~missing values~~, ~~wrong data type~~
- [x]  purpose: ~~duplicates~~, ~~categorize & extract~~

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
We checked the categorization of our data in sufficient detail 👍
    
</div>

## 3 Answer these questions

\* default rate  = proportion of customers who failed to respect a previous loan's conditions.

- Is there a relation between having kids and repaying a loan on time?
***

In [90]:
debt_children = df[['children', 'debt']].reset_index() \
    .pivot_table(
        index='children',
        columns='debt',
        values='index',
        aggfunc='count',
        margins=True
    ).fillna(0).astype('int').reset_index()
debt_children['default rate (%)'] = debt_children[True] / debt_children['All'] * 100
debt_children = debt_children.reindex(debt_children.sort_values(debt_children.columns[-1]).index)
debt_children

debt,children,False,True,All,default rate (%)
5,5,9,0,9,0.000000
0,0,13086,1063,14149,7.512898
6,All,19784,1741,21525,8.088269
3,3,303,27,330,8.181818
1,1,4420,445,4865,9.146968
2,2,1929,202,2131,9.479118
4,4,37,4,41,9.756098


### 3.1 Conclusion

In [91]:
(9/21252 * 100)

0.042348955392433656

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Good 👍
    
</div>

Out of 9 customers with 5 children, none defaulted, but they are statistically insignificant (0.04%).  
In general, customers with $\color{red}{\text{children}}$ have a $\color{red}{\text{higher default rate}}$ than those without children.

- Is there a relation between marital status and repaying a loan on time?
***

In [92]:
family_status = df[['family_status_id', 'debt']] \
    .join(family_status_ref, on='family_status_id') \
    .pivot_table(
        index='family_status',
        columns='debt',
        values='family_status_id',
        aggfunc='count',
        margins=True
    ).reset_index()
family_status['default rate (%)'] = family_status[True] / family_status['All'] * 100
family_status = family_status.reindex(family_status.sort_values(family_status.columns[-1]).index)
family_status

debt,family_status,False,True,All,default rate (%)
4,widow / widower,897,63,960,6.562500
1,divorced,1110,85,1195,7.112971
2,married,11449,931,12380,7.520194
5,All,19784,1741,21525,8.088269
0,civil partnership,3789,388,4177,9.288963
3,unmarried,2539,274,2813,9.740491


<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Good 👍
    
Please note that our actions are repeated in terms of calculations.
    
This suggests that we can create a function for constant calculations. Thus, we can optimize our code 😊
    
</div>

### 3.2 Conclusion

Customers who are either in $\color{red}{\text{civil partnership}}$ or $\color{red}{\text{unmarried}}$ have a $\color{red}{\text{higher default rate}}$.

- Is there a relation between income level and repaying a loan on time?

In [93]:
def debt_vs_income(bins):
    debt_income = pd.concat([
        df['debt'],
        pd.cut(
            df['income'],
            bins=bins,
            right=False
        ).rename('income_range')
    ], axis=1, sort=True).reset_index() \
    .pivot_table(
        index='income_range',
        columns='debt',
        values='index',
        aggfunc='count',
        margins=True
    ).reset_index()

    debt_income['default rate (%)'] = debt_income[True] / debt_income['All'] * 100
    debt_income = debt_income.reindex(debt_income.sort_values(debt_income.columns[-1]).index)
    display(debt_income)

In [94]:
debt_vs_income(bins=[0, 20000, 40000, 100000])

debt,income_range,False,True,All,default rate (%)
2,"[40000, 100000)",2525,188,2713,6.929598
3,All,19691,1735,21426,8.097638
0,"[0, 20000)",6977,622,7599,8.185288
1,"[20000, 40000)",10189,925,11114,8.322836


In [95]:
debt_vs_income(bins=[0, 10000, 20000, 30000, 40000, 50000, 60000, 70000])

debt,income_range,False,True,All,default rate (%)
6,"[60000, 70000)",278,16,294,5.442177
0,"[0, 10000)",868,58,926,6.263499
4,"[40000, 50000)",1390,102,1492,6.836461
3,"[30000, 40000)",2992,247,3239,7.625810
7,All,19428,1719,21147,8.128813
5,"[50000, 60000)",594,54,648,8.333333
1,"[10000, 20000)",6109,564,6673,8.451971
2,"[20000, 30000)",7197,678,7875,8.609524


<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Good 👍
    
Including it was possible to count using the `groupby` method, we will mean it as an alternative    
</div>

### 3.3 Conclusion

In general, $\color{green}{\text{higher}}$ income appears linearly related to $\color{green}{\text{lower default rate}}$.

- How do different loan purposes affect on-time repayment of the loan?

In [96]:
debt_purpose = df[['purpose_id', 'debt']] \
    .join(purpose_ref, on='purpose_id') \
    .pivot_table(
        index='purpose',
        columns='debt',
        values='purpose_id',
        aggfunc='count',
        margins=True
    ).reset_index()
debt_purpose['default rate (%)'] = debt_purpose[True] / debt_purpose['All'] * 100
debt_purpose = debt_purpose.reindex(debt_purpose.sort_values(debt_purpose.columns[-1]).index)
debt_purpose

debt,purpose,False,True,All,default rate (%)
1,home,3564,256,3820,6.701571
2,real-estate,6494,526,7020,7.492877
4,wedding,2162,186,2348,7.921635
5,All,19784,1741,21525,8.088269
0,education,3652,370,4022,9.199403
3,vehicle,3912,403,4315,9.339513


### 3.4 Conclusion

Customers who request a loan for $\color{red}{\text{education}}$ purposes, or in order to pay for a $\color{red}{\text{vehicle}}$,    
  appear to have a $\color{red}{\text{higher default rate}}$ compared to other customers.

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
Good 👍
    
Including it would be possible to build graphs, for example, using the `hist` method
    
Just the ways of visualization, we will already go through in the next project    
</div>

## 4. General conclusion

Review: the purpose of this report was to answer the question:
> Does a customer’s marital status and number of children have an impact on whether they will default on a loan?

By analysing the data it was shown that there is in fact a connection between marital status and number of children to whether a customer default on a loan in the past.

$\color{green}{\text{low-risk}}$ customers:
- no children (-0.5)
- widowes/widoers (-1.5), divorced (-1)
- high income (-0.5 to -2.5)
- purposes: renovation (-1.3), real-estate (-0.5)

$\color{red}{\text{high-risk}}$ customers:
- having children (1 to 2)
- civil partnership (1.2), unmarried (1.7)
- low income (0.5)
- purposes: education (1.1), vehicle (1.3)

\* numbers represent default rate below (-) or above (+) average

Data should also be plotted for visualization,  
and more proccessed more rigorously by statistical methods before drawing any final conclusions.

<div class="alert alert-success"> <b>Reviewer comment:</b> 
    
The conclusions are clear and logical, and most importantly, they are supported by the revealed facts.
       
In general, the project itself is at a good level! Throughout the entire work, the depth of the task development is felt and this is an absolute plus for us.
    
I would like to note that we have learned a lot of new things in this project and we can confidently move on. What I recommend to pay attention to right from the start, so to speak, namely, the repetition of the code. If our actions begin to be repeated more than 2 times, then it makes sense to think about:
    
- implementation of the cycle
- implementation of the function
- the introduction of a combination of cycle + function
    
Such things may not immediately catch the eye, but gradually we will come to this approach
    
Congratulations on the successful completion of the project 😊👍
    
And I wish you success in your new work 😊
    
**[general comment]** Do not forget about the design of your work, this is an equally important stage. I recommend to “feel free” to use cells like **markdown** where we need to display our thoughts, including using additional styling.
    
https://sqlbak.com/blog/jupyter-notebook-markdown-cheatsheet
    
</div>

## Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.